In [1]:
# Importe les modules
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 200)

# Projet 8 : Préparation des données

## Sommaire
* [Introduction](#1)
* [Objectif](#2)
* [Préparation des données](#3)
    * [Fichier annuaire](#3-1)
    * [Fichier des IPS](#3-2)
    * [Fichier des indicateurs de résulats](#3-3)

## Introduction<a id='1'></a>

L'ensemble des données proviennent du Ministère de l'Education Nationale et ont été récupérées sur la plateforme [data.education.gouv.fr](https://data.education.gouv.fr/pages/accueil/). Les fichiers suivant ont été téléchargés au format CSV :
* `fr-en-annuaire-education` ;
* `fr-en-ips-lycees-ap2022` ;
* `fr-en-ips_lycees` ;
* `fr-en-indicateurs-de-resultat-des-lycees-denseignement-general-et-technologique` ;
* `fr-en-indicateurs-de-resultat-des-lycees-denseignement-professionnels`.

Tous ces fichiers ont été mis à jour en 2023 et ils contiennent des données jusqu'en 2022.

Un fichier supplémentaire, permettant à Power BI de créer des visuels de type « Carte de formes », a été généré à partir de [FranceGEOJSON](https://france-geojson.gregoiredavid.fr/).
> Le projet Github france-geojson propose au format GeoJSON les cartes des régions, départements, arrondissements, cantons et communes de France (métropole et départements d'outre-mer) à partir des données publiées par l'IGN et l'INSEE.

Le fichier `geojson` a ensuite été converti au format `json` grâce au site [Mapshaper](https://mapshaper.org/) pour être utilisable par Power BI. Il se nomme : `fr-departements-shape-map`.

## Objectif<a id='2'></a>

Dans ce *notebook*, nous allons préparer les données qui vont être utilisées au sein de Power BI en :
* concaténant les différents fichiers des IPS , tout comme ceux des indicateurs de résultats (IVAL) ;
* vérifiant la cohérence géographique et temporelle des données entre les fichiers ;
* renommant ou supprimant de variables ;
* vérifiant les types, les doublons, les valeurs manquantes, etc..

## Préparation des données<a id='3'></a>

On commence par créer un fonction permettant de formater les variables des différents fichiers que nous allons importer. Cela facilitera les manipulations sur les *DataFrames*.

In [2]:
def snake_case (df):
    """
    Converti les noms de variables d'un DataFrame au format Snake Case :
    - mise en minuscule ;
    - remplacement de l'espaces, du tiret et de l'apostrophe par '_' ;
    - suppression des accents.
    """
    df.columns = (
        df.columns
        .str.lower()
        .str.replace('\'', '_')
        .str.replace(' - ', '_')
        .str.replace('-', '_')
        .str.replace(' ', '_')
        .str.normalize('NFKD')
        .str.encode('ascii', errors='ignore')
        .str.decode('utf-8')
    )

### Fichier annuaire<a id='3-1'></a>

Grâce à ce fichier, nous allons récupérer les positions géographiques des établissements.

In [3]:
df_annuaire = pd.read_csv('./datasets/raw/fr-en-annuaire-education.csv', sep=';', low_memory=False)
snake_case(df_annuaire)

In [4]:
df_annuaire.head(3)

,identifiant_de_l_etablissement,nom_etablissement,type_etablissement,statut_public_prive,adresse_1,adresse_2,adresse_3,code_postal,code_commune,nom_commune,code_departement,code_academie,code_region,ecole_maternelle,ecole_elementaire,voie_generale,voie_technologique,voie_professionnelle,telephone,fax,web,mail,restauration,hebergement,ulis,apprentissage,segpa,section_arts,section_cinema,section_theatre,section_sport,section_internationale,section_europeenne,lycee_agricole,lycee_militaire,lycee_des_metiers,post_bac,appartenance_education_prioritaire,greta,siren_siret,nombre_d_eleves,fiche_onisep,position,type_contrat_prive,libelle_departement,libelle_academie,libelle_region,coordonnee_x,coordonnee_y,epsg,nom_circonscription,latitude,longitude,precision_localisation,date_ouverture,date_maj_ligne,etat,ministere_tutelle,etablissement_multi_lignes,rpi_concentre,rpi_disperse,code_nature,libelle_nature,code_type_contrat_prive,pial,etablissement_mere,type_rattachement_etablissement_mere,code_bassin_formation,libelle_bassin_formation
0,0641141T,ECOLE PRIMAIRE PUBLIQUE HERAURITZ,Ecole,Public,Quartier Herauritz,NaN,64480 USTARITZ,64480,64547,Ustaritz,064,4,75,1.0,1.0,NaN,NaN,NaN,0559930682,NaN,NaN,ce.0641141T@ac-bordeaux.fr,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21640547200038,99.0,NaN,"43.41939759772092, -1.4672331951877942",SANS OBJET,Pyrénées-Atlantiques,Bordeaux,Nouvelle-Aquitaine,338242.9,6268045.1,EPSG:2154,Circonscription d'inspection du 1er degré de B...,43.419398,-1.467233,PLAQUE_ADRESSE,1970-02-04,2023-06-13,OUVERT,MINISTERE DE L'EDUCATION NATIONALE,0,0.0,NaN,151,ECOLE DE NIVEAU ELEMENTAIRE,99,0641392R,NaN,NaN,04503,PAYS BASQUE
1,0641147Z,ECOLE PRIMAIRE DU BOURG,Ecole,Public,BOURG,NaN,64130 VIODOS ABENSE DE BAS,64130,64559,Viodos-Abense-de-Bas,064,4,75,1.0,1.0,NaN,NaN,NaN,0559191405,NaN,NaN,ce.0641147Z@ac-bordeaux.fr,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21640559700024,38.0,NaN,"43.24304294866094, -0.8803405181620836",SANS OBJET,Pyrénées-Atlantiques,Bordeaux,Nouvelle-Aquitaine,384765.4,6245952.5,EPSG:2154,Circonscription d'inspection du 1er degré de S...,43.243043,-0.880341,PLAQUE_ADRESSE,1970-02-06,2023-06-13,OUVERT,MINISTERE DE L'EDUCATION NATIONALE,0,0.0,NaN,151,ECOLE DE NIVEAU ELEMENTAIRE,99,0640039V,NaN,NaN,NaN,NaN
2,0641170Z,ECOLE ELEMENTAIRE PUBLIQUE,Ecole,Public,Rue de l'Ardoisière,NaN,64490 BEDOUS,64490,64104,Bedous,064,4,75,0.0,1.0,NaN,NaN,NaN,0559347476,NaN,NaN,ce.0641170Z@ac-bordeaux.fr,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21640104200033,57.0,NaN,"42.99894867732615, -0.5992616889748449",SANS OBJET,Pyrénées-Atlantiques,Bordeaux,Nouvelle-Aquitaine,406346.9,6217760.6,EPSG:2154,Circonscription d'inspection du 1er degré d'Ol...,42.998949,-0.599262,PLAQUE_ADRESSE,1970-06-03,2023-06-13,OUVERT,MINISTERE DE L'EDUCATION NATIONALE,0,0.0,064411,151,ECOLE DE NIVEAU ELEMENTAIRE,99,0640016V,NaN,NaN,04502,CENTRE


In [5]:
print(f'Dimensions du DataFrame : {df_annuaire.shape}')

Dimensions du DataFrame : (68962, 69)


In [6]:
df_annuaire['type_etablissement'].value_counts()

Ecole                         49973
Collège                        7649
Lycée                          5746
Médico-social                  2274
Service Administratif          2155
Information et orientation      426
Autre                           375
EREA                             79
Name: type_etablissement, dtype: int64

Le fichier compte de nombreux enregistrements. Plusieurs types d'établissements sont présents mais seuls les lycées nous intéressent. On supprime les autres.

In [7]:
df_annuaire = df_annuaire.loc[df_annuaire['type_etablissement'] == 'Lycée'].reset_index()

Le fichier comprend également beaucoup de variables qui ne nous seront pas utiles car seules les positions nous intéressent.

In [8]:
df_annuaire = df_annuaire[
    [
        'identifiant_de_l_etablissement',
        'coordonnee_x',
        'coordonnee_y',
        'latitude',
        'longitude'
    ]
]

On renomme la variable de l'identifiant.

In [9]:
df_annuaire = df_annuaire.rename(
    columns={'identifiant_de_l_etablissement': 'uai'})

Des établissements peuvent apparaître plusieurs fois lorsqu'ils proposent plusieurs filières, on supprime donc les doublons.

In [10]:
print(f' Il y a {df_annuaire.duplicated().sum()} établissements en doublon.')

 Il y a 69 établissements en doublon.


In [11]:
df_annuaire = df_annuaire.drop_duplicates(keep='first')

Voyons le nombre d'établissements pour lesquels nous n'avons pas les coordonnées.

In [12]:
df_annuaire.isna().sum()

uai              0
coordonnee_x    78
coordonnee_y    78
latitude        78
longitude       78
dtype: int64

Enfin on vérifie le type des données.

In [13]:
df_annuaire.dtypes

uai              object
coordonnee_x    float64
coordonnee_y    float64
latitude        float64
longitude       float64
dtype: object

On exporte en CSV.

In [14]:
df_annuaire.to_csv('./datasets/position-geographique-lycees.csv',
                    index=False, decimal=',', quoting=1)

In [15]:
del(df_annuaire)

### Fichiers des IPS<a id='3-2'></a>

In [16]:
df_ips_2022 = pd.read_csv('./datasets/raw/fr-en-ips-lycees-ap-2022.csv', sep=';')
snake_case(df_ips_2022)

df_ips_2017_2021 = pd.read_csv('./datasets/raw/fr-en-ips-lycees-2017-2021.csv', sep=';')
snake_case(df_ips_2017_2021)

In [17]:
display(df_ips_2022.head(3))
display(df_ips_2017_2021.head(3))

,rentree_scolaire,academie,code_du_departement,departement,uai,nom_de_l_etablissment,code_insee_de_la_commune,nom_de_la_commune,secteur,type_de_lycee,effectifs_voie_gt,effectifs_voie_pro,effectifs_ensemble_gt_pro,ips_voie_gt,ips_voie_pro,ips_ensemble_gt_pro,ecart_type_de_l_ips_voie_gt,ecart_type_de_l_ips_voie_pro
0,2022-2023,LYON,001,AIN,0010001W,LYCEE PROFESSIONNEL ALEXANDRE BERARD LYCEE DES...,01004,AMBERIEU EN BUGEY,public,LP,NaN,583.0,583.0,NaN,91.5,91.5,NaN,27.2
1,2022-2023,LYON,001,AIN,0010006B,LYCEE POLYVALENT SAINT EXUPERY LYCEE DES METIE...,01033,VALSERHONE,public,LPO,556.0,334.0,890.0,99.2,84.2,93.6,33.3,24.8
2,2022-2023,LYON,001,AIN,0010013J,LYCEE GENERAL LALANDE,01053,BOURG EN BRESSE,public,LEGT,1028.0,NaN,1028.0,123.0,NaN,123.0,34.7,NaN


,rentree_scolaire,academie,code_du_departement,departement,uai,nom_de_l_etablissment,code_insee_de_la_commune,nom_de_la_commune,secteur,type_de_lycee,ips_voie_gt,ips_voie_pro,ips_ensemble_gt_pro,ecart_type_de_l_ips_voie_gt,ecart_type_de_l_ips_voie_pro
0,2018-2019,CRETEIL,77.0,SEINE-ET-MARNE,0771995A,LYCEE PROFESSIONNEL LE CHAMP DE CLAYE LYCEE DE...,77118.0,CLAYE SOUILLY,public,LP,NaN,87.4,87.4,NaN,NaN
1,2018-2019,CRETEIL,77.0,SEINE-ET-MARNE,0772127U,LYCEE GENERAL ET TECHNOLOGIQUE GALILEE,77122.0,COMBS LA VILLE,public,LEGT,115.6,NaN,115.6,NaN,NaN
2,2018-2019,CRETEIL,77.0,SEINE-ET-MARNE,0772151V,LYCEE PROFESSIONNEL PRIVE LES SINOPLIES,77390.0,ROISSY EN BRIE,privé sous contrat,LP,NaN,114.2,114.2,NaN,NaN


In [18]:
print(
    f'Dimensions des DataFrames :\n'
    f'- df_ips_2022 : {df_ips_2022.shape}\n'
    f'- df_ips_2017_2021 : {df_ips_2017_2021.shape}'
    )

Dimensions des DataFrames :
- df_ips_2022 : (3598, 18)
- df_ips_2017_2021 : (21777, 15)


Le nombre de colonnes diffère, la dernière version contient plus de variables.

In [19]:
df_ips_2022.columns

Index(['rentree_scolaire', 'academie', 'code_du_departement', 'departement',
       'uai', 'nom_de_l_etablissment', 'code_insee_de_la_commune',
       'nom_de_la_commune', 'secteur', 'type_de_lycee', 'effectifs_voie_gt',
       'effectifs_voie_pro', 'effectifs_ensemble_gt_pro', 'ips_voie_gt',
       'ips_voie_pro', 'ips_ensemble_gt_pro', 'ecart_type_de_l_ips_voie_gt',
       'ecart_type_de_l_ips_voie_pro'],
      dtype='object')

In [20]:
df_ips_2017_2021.columns

Index(['rentree_scolaire', 'academie', 'code_du_departement', 'departement',
       'uai', 'nom_de_l_etablissment', 'code_insee_de_la_commune',
       'nom_de_la_commune', 'secteur', 'type_de_lycee', 'ips_voie_gt',
       'ips_voie_pro', 'ips_ensemble_gt_pro', 'ecart_type_de_l_ips_voie_gt',
       'ecart_type_de_l_ips_voie_pro'],
      dtype='object')

Les effectifs ont été ajoutés en 2022. On les supprime et on supprime également les colonnes des écarts-type.

In [21]:
df_ips_2022 = df_ips_2022.drop(
    columns=['effectifs_voie_gt', 'effectifs_voie_pro',
             'effectifs_ensemble_gt_pro', 'ecart_type_de_l_ips_voie_gt',
             'ecart_type_de_l_ips_voie_pro']
)

df_ips_2017_2021 = df_ips_2017_2021.drop(
    columns=['ecart_type_de_l_ips_voie_gt', 'ecart_type_de_l_ips_voie_pro']
)

In [22]:
df_ips_2022.dtypes

rentree_scolaire             object
academie                     object
code_du_departement          object
departement                  object
uai                          object
nom_de_l_etablissment        object
code_insee_de_la_commune     object
nom_de_la_commune            object
secteur                      object
type_de_lycee                object
ips_voie_gt                 float64
ips_voie_pro                float64
ips_ensemble_gt_pro         float64
dtype: object

In [23]:
df_ips_2017_2021.dtypes

rentree_scolaire             object
academie                     object
code_du_departement         float64
departement                  object
uai                          object
nom_de_l_etablissment        object
code_insee_de_la_commune    float64
nom_de_la_commune            object
secteur                      object
type_de_lycee                object
ips_voie_gt                 float64
ips_voie_pro                float64
ips_ensemble_gt_pro         float64
dtype: object

On convertit une partie des variables.

In [24]:
df_ips_2022 = df_ips_2022.astype(
    {
        'rentree_scolaire': 'category',
        'academie': 'category',
        'code_du_departement': 'category',
        'departement': 'category',
        'uai': 'category',
        'code_insee_de_la_commune': 'category',
        'secteur': 'category',
        'type_de_lycee': 'category'
    }
)

df_ips_2017_2021 = df_ips_2017_2021.astype(
    {
        'rentree_scolaire': 'category',
        'academie': 'category',
        'code_du_departement': 'category',
        'departement': 'category',
        'uai': 'category',
        'code_insee_de_la_commune': 'category',
        'secteur': 'category',
        'type_de_lycee': 'category'
    }
)

Et on vérifie les valeurs manquantes

In [25]:
df_ips_2022.isna().sum()

rentree_scolaire               0
academie                       0
code_du_departement            0
departement                    0
uai                            0
nom_de_l_etablissment          0
code_insee_de_la_commune       0
nom_de_la_commune              0
secteur                        0
type_de_lycee                  0
ips_voie_gt                 1134
ips_voie_pro                1523
ips_ensemble_gt_pro            0
dtype: int64

Il y en a dans les IPS mais un lycée professionnel ne peut pas avoir de valeurs d'IPS correspondant à la voie générale et inversement. On constate d'ailleurs que « ips_ensemble_gt_pro » n'a aucune valeur manquante.

In [26]:
df_ips_2017_2021.isna().sum()

rentree_scolaire               0
academie                       0
code_du_departement           90
departement                    0
uai                            0
nom_de_l_etablissment          0
code_insee_de_la_commune      90
nom_de_la_commune              0
secteur                        0
type_de_lycee                  0
ips_voie_gt                 6985
ips_voie_pro                9221
ips_ensemble_gt_pro            0
dtype: int64

On constate de nouveau les valeurs manquantes des IPS, par contre on a 90 valeurs manquantes dans « code_du_departement » et « code_insee_de_la_commune ».

In [27]:
df_ips_2017_2021.loc[df_ips_2017_2021['code_du_departement'].isna()].head(3)

,rentree_scolaire,academie,code_du_departement,departement,uai,nom_de_l_etablissment,code_insee_de_la_commune,nom_de_la_commune,secteur,type_de_lycee,ips_voie_gt,ips_voie_pro,ips_ensemble_gt_pro
143,2018-2019,CORSE,NaN,CORSE-DU-SUD,6200001G,LYCEE GENERAL ET TECHNOLOGIQUE FESCH AJACCIO,NaN,AJACCIO,public,LEGT,115.7,NaN,115.7
910,2019-2020,CORSE,NaN,CORSE-DU-SUD,6200650M,INSTITUTION SAINT PAUL (LYCEE GENERAL PRIVE) A...,NaN,AJACCIO,privé sous contrat,LEGT,113.6,NaN,113.6
911,2019-2020,CORSE,NaN,HAUTE-CORSE,7200093N,LYCEE PROFESSIONNEL JEAN NICOLI BASTIA,NaN,BASTIA,public,LP,NaN,85.3,85.3


In [28]:
nombre_acad = df_ips_2017_2021.loc[
    df_ips_2017_2021['code_du_departement'].isna(), 'academie'].nunique()

print(f'Il y a des départements non renseignés dans {nombre_acad} académie(s)')

del nombre_acad

Il y a des départements non renseignés dans 1 académie(s)


Les valeurs sont manquantes uniquement pour l'académie de Corse. On crée un dictionnaire à partir de du fichier de 2022 et de la variables « uai », puis on utilise ce dernier pour compléter les valeurs dans le fichier de 2017 à 2022 en veillant à respecter le format d'origine du fichier (« 0XX.0 »).

In [29]:
# Remplissage du code département
temp_dict = dict(zip(df_ips_2022['uai'], df_ips_2022['code_du_departement']))
df_ips_2017_2021['code_du_departement'] = (
    df_ips_2017_2021['code_du_departement']
    .astype('string')
    .fillna(df_ips_2017_2021['uai'].map(temp_dict).str[1:] + '.0')
)

# Remplissage du code commune
temp_dict = dict(zip(df_ips_2022['uai'], df_ips_2022['code_insee_de_la_commune']))
df_ips_2017_2021['code_insee_de_la_commune'] = (
    df_ips_2017_2021['code_insee_de_la_commune']
    .astype('string')
    .fillna(df_ips_2017_2021['uai'].map(temp_dict) + '.0')
)

del temp_dict

On va maintenant harmoniser le format des valeurs de « code_du_departement » et « code_insee_de_la_commune ». On souhaite obtenir :
* le code département avec deux caractères, à l'exception des DROM-COM qui en compte trois ;
* le code de la commune avec cinq caractères.

En fonction de la source des données, le traitement sera différents.
|                          | `df_ips_2017_2021`                             | `df_ips_2022`          |
|--------------------------|:----------------------------------------------:|:----------------------:|
|`code_du_departement`     |Suppression décimale<br>Format deux caractères  |Format deux caractères  |
|`code_insee_de_la_commune`|Format deux caractères                          |Ok                      |

In [30]:
df_ips_2017_2021['code_du_departement'] = (
    df_ips_2017_2021['code_du_departement']
    .astype('string')
    .str[:-2] # Suppression décimale
    .apply(lambda x: '0' + x if len(x) == 1 else x) # Format 2 caractères
)

df_ips_2017_2021['code_insee_de_la_commune'] = (
    df_ips_2017_2021['code_insee_de_la_commune']
    .astype('string')
    .str[:-2] # Suppression décimale
)

df_ips_2022['code_du_departement'] = (
    df_ips_2022['code_du_departement']
    .astype('string')
    .apply(lambda x: x.strip()[1:] if x.startswith('0') else x) # Format 2 caractères
)

On concatène les deux *DataFrames* et on exporte le résultat en CSV. Comme l'étude porte sur la France métropolitaine on supprime également les enregistrements liés aux DROM-COM.

In [31]:
# Concatène les données
df_temp = pd.concat([df_ips_2017_2021, df_ips_2022]).reset_index(drop=True)

# Met en majuscule le premier mot de "secteur"
df_temp['secteur'] = df_temp['secteur'].str.capitalize().astype('category')

# Crée la liste des départements à supprimer
liste_drom_com = list(
    df_temp.loc[
        (df_temp['code_du_departement'].str.startswith('9'))
        & (df_temp['code_du_departement'].str.len() == 3)
    ]['code_du_departement'].unique()
)

# Supprime les DROM-COM
df_temp = df_temp.loc[~df_temp['code_du_departement'].isin(liste_drom_com)]

# Exporte en CSV
df_temp.to_csv('./datasets/ips-lycees-2017-2022-hors-drom-com.csv',
                    index=False, decimal=',', quoting=1)


In [32]:
del(df_ips_2017_2021, df_ips_2022, df_temp, liste_drom_com)

### Fichiers des indicateurs de résultats<a id='3-3'></a>

In [33]:
df_indicateurs_gt = pd.read_csv(
    './datasets/raw/fr-en-indicateurs-resultat-lycees-generaux-technologiques-2012-2022.csv'
    , sep=';', low_memory=False)
snake_case(df_indicateurs_gt)

df_indicateurs_pro = pd.read_csv(
    './datasets/raw/fr-en-indicateurs-resultat-lycees-professionnels-2012-2022.csv'
    , sep=';', low_memory=False)
snake_case(df_indicateurs_pro)

In [34]:
display(df_indicateurs_gt.head(3))
display(df_indicateurs_pro.head(3))

,etablissement,annee,ville,uai,code_commune,academie,departement,secteur,presents_l,presents_es,presents_s,presents_stg,presents_sti2d,presents_std2a,presents_stmg,presents_sti,presents_stl,presents_st2s,presents_tmd,presents_sthr,presents_toutes_series,taux_de_reussite_l,taux_de_reussite_es,taux_de_reussite_s,taux_de_reussite_stg,taux_de_reussite_sti2d,taux_de_reussite_std2a,taux_de_reussite_stmg,taux_de_reussite_sti,taux_de_reussite_stl,taux_de_reussite_st2s,taux_de_reussite_tmd,taux_de_reussite_sthr,taux_de_reussite_toutes_series,taux_de_reussite_attendu_acad_l,taux_de_reussite_attendu_acad_es,taux_de_reussite_attendu_acad_s,taux_de_reussite_attendu_acad_stg,taux_de_reussite_attendu_acad_sti2d,taux_de_reussite_attendu_acad_std2a,taux_de_reussite_attendu_acad_stmg,taux_de_reussite_attendu_acad_sti,taux_de_reussite_attendu_acad_stl,taux_de_reussite_attendu_acad_st2s,taux_de_reussite_attendu_acad_tmd,taux_de_reussite_attendu_acad_sthr,taux_de_reussite_attendu_acad_toutes_series,taux_de_reussite_attendu_france_l,taux_de_reussite_attendu_france_es,taux_de_reussite_attendu_france_s,taux_de_reussite_attendu_france_stg,taux_de_reussite_attendu_france_sti2d,taux_de_reussite_attendu_france_std2a,taux_de_reussite_attendu_france_stmg,taux_de_reussite_attendu_france_sti,taux_de_reussite_attendu_france_stl,taux_de_reussite_attendu_france_st2s,taux_de_reussite_attendu_france_tmd,taux_de_reussite_attendu_france_sthr,taux_de_reussite_attendu_france_toutes_series,taux_de_mentions_l,taux_de_mentions_es,taux_de_mentions_s,taux_de_mentions_sti2d,taux_de_mentions_std2a,taux_de_mentions_stmg,taux_de_mentions_stl,taux_de_mentions_st2s,taux_de_mentions_tmd,taux_de_mentions_sthr,taux_de_mentions_toutes_series,taux_de_mentions_attendu_l,taux_de_mentions_attendu_es,taux_de_mentions_attendu_s,taux_de_mentions_attendu_sti2d,taux_de_mentions_attendu_std2a,taux_de_mentions_attendu_stmg,taux_de_mentions_attendu_stl,taux_de_mentions_attendu_st2s,taux_de_mentions_attendu_tmd,taux_de_mentions_attendu_sthr,taux_de_mentions_attendu_toutes_series,structure_pedagogique_5_groupes,structure_pedagogique_7_groupes,pourcentage_bacheliers_sortants_2de_1re_terminale_etablissement,pourcentage_bacheliers_sortants_terminales_etablissement,pourcentage_bacheliers_sortants_2de_1re_terminale_acad,pourcentage_bacheliers_sortants_terminale_acad,pourcentage_bacheliers_sortants_2de_1re_terminale_france,pourcentage_bacheliers_sortants_terminale_france,effectif_de_seconde,effectif_de_premiere,effectif_de_terminale,taux_d_acces_2nde_bac,taux_d_acces_attendu_acad_2nde_bac,taux_d_acces_attendu_france_2nde_bac,taux_d_acces_1ere_bac,taux_d_acces_attendu_acad_1ere_bac,taux_d_acces_attendu_france_1ere_bac,taux_d_acces_terminale_bac,taux_d_acces_attendu_france_terminale_bac,region,code_region,code_departement,libelle_departement,valeur_ajoutee_du_taux_de_reussite_toutes_series,valeur_ajoutee_du_taux_d_acces_2nde_bac,valeur_ajoutee_du_taux_de_mentions_toutes_series,valeur_ajoutee_du_taux_de_reussite_l,valeur_ajoutee_du_taux_de_reussite_es,valeur_ajoutee_du_taux_de_reussite_s,valeur_ajoutee_du_taux_de_reussite_stg,valeur_ajoutee_du_taux_de_reussite_sti2d,valeur_ajoutee_du_taux_de_reussite_std2a,valeur_ajoutee_du_taux_de_reussite_stmg,valeur_ajoutee_du_taux_de_reussite_sti,valeur_ajoutee_du_taux_de_reussite_stl,valeur_ajoutee_du_taux_de_reussite_st2s,valeur_ajoutee_du_taux_de_reussite_tmd,valeur_ajoutee_du_taux_de_reussite_sthr,valeur_ajoutee_du_taux_d_acces_1ere_bac,valeur_ajoutee_du_taux_d_acces_terminale_bac,valeur_ajoutee_du_taux_de_mentions_l,valeur_ajoutee_du_taux_de_mentions_es,valeur_ajoutee_du_taux_de_mentions_s,valeur_ajoutee_du_taux_de_mentions_sti2d,valeur_ajoutee_du_taux_de_mentions_std2a,valeur_ajoutee_du_taux_de_mentions_stmg,valeur_ajoutee_du_taux_de_mentions_stl,valeur_ajoutee_du_taux_de_mentions_st2s,valeur_ajoutee_du_taux_de_mentions_tmd,valeur_ajoutee_du_taux_de_mentions_sthr,presents_gnle,taux_de_reussite_gnle,valeur_ajoutee_du_taux_de_reussite_gnle,taux_de_mentions_gnle,v

,etablissement,uai,annee,ville,code_commune,academie,departement,secteur,presents_specialites_pluri_technologiques_de_la_production,presents_transformations,presents_genie_civil_construction_bois,presents_materiaux_souples,presents_mecanique_electricite_electronique,presents_production,presents_specialites_plurivalentes_des_services,presents_echanges_et_gestion,presents_communication_et_information,presents_services_aux_personnes,presents_services_a_la_collectivite,presents_services,presents_toutes_series,taux_de_reussite_specialites_pluri_technologiques_de_la_production,taux_de_reussite_transformations,taux_de_reussite_genie_civil_construction_bois,taux_de_reussite_materiaux_souples,taux_de_reussite_mecanique_electricite_electronique,taux_de_reussite_production,taux_de_reussite_specialites_plurivalentes_des_services,taux_de_reussite_echanges_et_gestion,taux_de_reussite_communication_et_information,taux_de_reussite_services_aux_personnes,taux_de_reussite_services_a_la_collectivite,taux_de_reussite_services,taux_de_reussite_toutes_series,taux_de_reussite_attendu_france_specialites_pluri_technologiques_de_la_production,taux_de_reussite_attendu_france_transformations,taux_de_reussite_attendu_france_genie_civil_construction_bois,taux_de_reussite_attendu_france_materiaux_souples,taux_de_reussite_attendu_france_mecanique_electricite_electronique,taux_de_reussite_attendu_france_specialites_plurivalentes_des_services,taux_de_reussite_attendu_france_echanges_et_gestion,taux_de_reussite_attendu_france_communication_et_information,taux_de_reussite_attendu_france_services_aux_personnes,taux_de_reussite_attendu_france_services_a_la_collectivite,taux_de_reussite_attendu_acad_production,taux_de_reussite_attendu_acad_services,taux_de_reussite_attendu_acad_toutes_series,taux_de_reussite_attendu_france_production,taux_de_reussite_attendu_france_services,taux_de_reussite_attendu_france_toutes_series,pourcentage_bacheliers_sortants_2de_1re_terminale_etablissement,pourcentage_bacheliers_sortants_terminales_etablissement,pourcentage_bacheliers_sortants_2de_1re_terminale_acad,pourcentage_bacheliers_sortants_terminale_acad,pourcentage_bacheliers_sortants_2de_1re_terminale_france,pourcentage_bacheliers_sortants_terminale_france,effectif_de_seconde,effectif_de_premiere,effectif_de_terminale,taux_d_acces_2nde_bac,taux_d_acces_attendu_acad_2nde_bac,taux_d_acces_attendu_france_2nde_bac,taux_d_acces_1ere_bac,taux_d_acces_attendu_acad_1ere_bac,taux_d_acces_attendu_france_1ere_bac,taux_d_acces_terminale_bac,taux_d_acces_attendu_france_terminale_bac,taux_de_mentions_specialites_pluri_technologiques_de_la_production,taux_de_mentions_transformations,taux_de_mentions_genie_civil_construction_bois,taux_de_mentions_materiaux_souples,taux_de_mentions_mecanique_electricite_electronique,taux_de_mentions_specialites_plurivalentes_des_services,taux_de_mentions_echanges_et_gestion,taux_de_mentions_communication_et_information,taux_de_mentions_services_aux_personnes,taux_de_mentions_services_a_la_collectivite,taux_de_mentions_production,taux_de_mentions_services,taux_de_mentions_toutes_series,taux_de_mentions_attendu_specialites_pluri_technologiques_de_la_production,taux_de_mentions_attendu_transformations,taux_de_mentions_attendu_genie_civil_construction_bois,taux_de_mentions_attendu_materiaux_souples,taux_de_mentions_attendu_mecanique_electricite_electronique,taux_de_mentions_attendu_specialites_plurivalentes_des_services,taux_de_mentions_attendu_echanges_et_gestion,taux_de_mentions_attendu_communication_et_information,taux_de_mentions_attendu_services_aux_personnes,taux_de_mentions_attendu_services_a_la_collectivite,taux_de_mentions_attendu_production,taux_de_mentions_attendu_services,taux_de_mentions_attendu_toutes_series,structure_pedagogique_7_groupes,region,code_region,libelle_departement,code_departement,valeur_ajoutee_du_taux_de_reussite_specialites_pluri_technologiques_de_la_production,valeur_ajoutee_du_taux_de_reussite_transformations,valeur_ajoutee_du_taux_de

In [35]:
print(
    f'Dimensions des DataFrames :\n'
    f'- df_indicateurs_gt : {df_indicateurs_gt.shape}\n'
    f'- df_indicateurs_pro : {df_indicateurs_pro.shape}'
    )

Dimensions des DataFrames :
- df_indicateurs_gt : (25477, 145)
- df_indicateurs_pro : (22227, 130)


On supprime les enregistrements datant d'avant 2017 (pour correspondre au fichiers des IPS) ainsi que ceux correspondants aux DROM-COM.

In [36]:
# Supprime les enregistrements antérieurs à 2017
df_indicateurs_gt = df_indicateurs_gt.loc[df_indicateurs_gt['annee'] >= 2017].reset_index()
df_indicateurs_pro = df_indicateurs_pro.loc[df_indicateurs_pro['annee'] >= 2017].reset_index()

# Crée la liste des départements à supprimer
liste_drom_com_gt = list(
    df_indicateurs_gt.loc[
        (df_indicateurs_gt['code_departement'].str.startswith('9'))
        & (df_indicateurs_gt['code_departement'].str.len() == 3)
    ]['code_departement'].unique()
)

liste_drom_com_pro = list(
    df_indicateurs_pro.loc[
        (df_indicateurs_pro['code_departement'].str.startswith('9'))
        & (df_indicateurs_pro['code_departement'].str.len() == 3)
    ]['code_departement'].unique()
)

# Supprime les DROM-COM
df_indicateurs_gt = df_indicateurs_gt.loc[
    ~df_indicateurs_gt['code_departement'].isin(liste_drom_com_gt)]

df_indicateurs_pro = df_indicateurs_pro.loc[
    ~df_indicateurs_pro['code_departement'].isin(liste_drom_com_pro)]

del(liste_drom_com_gt, liste_drom_com_pro)

Les tables contiennent énormément de variables dont nous n'aurons pas besoin, on ne va donc garder que celles qui nous intéressent.

In [37]:
df_indicateurs_gt = df_indicateurs_gt[
    [
        'annee',
        'uai',
        'presents_toutes_series',
        'taux_de_reussite_toutes_series',
        'valeur_ajoutee_du_taux_de_reussite_toutes_series',
        'taux_d_acces_2nde_bac',
        'valeur_ajoutee_du_taux_d_acces_2nde_bac',
        'taux_d_acces_1ere_bac',
        'valeur_ajoutee_du_taux_d_acces_1ere_bac',
        'taux_d_acces_terminale_bac',
        'valeur_ajoutee_du_taux_d_acces_terminale_bac',
        'taux_de_mentions_toutes_series',
        'valeur_ajoutee_du_taux_de_mentions_toutes_series'
    ]
]

df_indicateurs_pro = df_indicateurs_pro[
    [
        'annee',
        'uai',
        'presents_toutes_series',
        'taux_de_reussite_toutes_series',
        'valeur_ajoutee_du_taux_de_reussite_toutes_series',
        'taux_d_acces_2nde_bac',
        'valeur_ajoutee_du_taux_d_acces_2nde_bac',
        'taux_d_acces_1ere_bac',
        'valeur_ajoutee_du_taux_d_acces_1ere_bac',
        'taux_d_acces_terminale_bac',
        'valeur_ajoutee_du_taux_d_acces_terminale_bac',
        'taux_de_mentions_toutes_series',
        'valeur_ajoutee_du_taux_de_mentions_toutes_series'
    ]
]

On affiche les nouvelles dimensions des jeux de données.

In [38]:
print(
    f'Dimensions des DataFrames :\n'
    f'- df_indicateurs_gt : {df_indicateurs_gt.shape}\n'
    f'- df_indicateurs_pro : {df_indicateurs_pro.shape}'
    )

Dimensions des DataFrames :
- df_indicateurs_gt : (13332, 13)
- df_indicateurs_pro : (11540, 13)


On vérifie le type des variables.

In [39]:
display(df_indicateurs_gt.dtypes)
display(df_indicateurs_pro.dtypes)

annee                                                 int64
uai                                                  object
presents_toutes_series                                int64
taux_de_reussite_toutes_series                        int64
valeur_ajoutee_du_taux_de_reussite_toutes_series     object
taux_d_acces_2nde_bac                                object
valeur_ajoutee_du_taux_d_acces_2nde_bac              object
taux_d_acces_1ere_bac                               float64
valeur_ajoutee_du_taux_d_acces_1ere_bac              object
taux_d_acces_terminale_bac                          float64
valeur_ajoutee_du_taux_d_acces_terminale_bac         object
taux_de_mentions_toutes_series                      float64
valeur_ajoutee_du_taux_de_mentions_toutes_series     object
dtype: object

annee                                                 int64
uai                                                  object
presents_toutes_series                                int64
taux_de_reussite_toutes_series                        int64
valeur_ajoutee_du_taux_de_reussite_toutes_series     object
taux_d_acces_2nde_bac                               float64
valeur_ajoutee_du_taux_d_acces_2nde_bac              object
taux_d_acces_1ere_bac                               float64
valeur_ajoutee_du_taux_d_acces_1ere_bac              object
taux_d_acces_terminale_bac                          float64
valeur_ajoutee_du_taux_d_acces_terminale_bac         object
taux_de_mentions_toutes_series                      float64
valeur_ajoutee_du_taux_de_mentions_toutes_series     object
dtype: object

Et on les corrige/harmonise en utilisant `pd.Int16Dtype()` à la place d'utiliser simplement `Int16` car Python considère les valeurs nulles comme des « *float* ».

In [40]:
try:
    df_indicateurs_gt = df_indicateurs_gt.astype(
        {
        'annee': 'category',
        'uai': 'category',
        'presents_toutes_series': pd.Int16Dtype(),
        'taux_de_reussite_toutes_series': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_de_reussite_toutes_series': pd.Int16Dtype(),
        'taux_d_acces_2nde_bac': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_d_acces_2nde_bac': pd.Int16Dtype(),
        'taux_d_acces_1ere_bac': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_d_acces_1ere_bac': pd.Int16Dtype(),
        'taux_d_acces_terminale_bac': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_d_acces_terminale_bac': pd.Int16Dtype(),
        'taux_de_mentions_toutes_series': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_de_mentions_toutes_series': pd.Int16Dtype()
        }
    )
except ValueError as erreur:
    print(erreur)

invalid literal for int() with base 10: 'ND'


In [41]:
try:
    df_indicateurs_pro = df_indicateurs_pro.astype(
        {
        'annee': 'category',
        'uai': 'category',
        'presents_toutes_series': pd.Int16Dtype(),
        'taux_de_reussite_toutes_series': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_de_reussite_toutes_series': pd.Int16Dtype(),
        'taux_d_acces_2nde_bac': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_d_acces_2nde_bac': pd.Int16Dtype(),
        'taux_d_acces_1ere_bac': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_d_acces_1ere_bac': pd.Int16Dtype(),
        'taux_d_acces_terminale_bac': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_d_acces_terminale_bac': pd.Int16Dtype(),
        'taux_de_mentions_toutes_series': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_de_mentions_toutes_series': pd.Int16Dtype()
        }
    )
except ValueError as erreur:
    print(erreur)

invalid literal for int() with base 10: 'ND'


Certaines variables normalement numériques sont de type « object » car leur valeur est « ND ». C'est pour cela qu'on ne peut pas les convertir. Nous les remplaçons par « NaN » avant de refaire la conversion.

In [42]:
df_indicateurs_gt = df_indicateurs_gt.replace('ND', np.NaN)

try:
    df_indicateurs_gt = df_indicateurs_gt.astype(
        {
        'annee': 'category',
        'uai': 'category',
        'presents_toutes_series': pd.Int16Dtype(),
        'taux_de_reussite_toutes_series': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_de_reussite_toutes_series': pd.Int16Dtype(),
        'taux_d_acces_2nde_bac': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_d_acces_2nde_bac': pd.Int16Dtype(),
        'taux_d_acces_1ere_bac': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_d_acces_1ere_bac': pd.Int16Dtype(),
        'taux_d_acces_terminale_bac': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_d_acces_terminale_bac': pd.Int16Dtype(),
        'taux_de_mentions_toutes_series': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_de_mentions_toutes_series': pd.Int16Dtype()
        }
    )
except ValueError as erreur:
    print(erreur)

invalid literal for int() with base 10: '.'


In [43]:
df_indicateurs_pro = df_indicateurs_pro.replace('ND', np.NaN)

try:
    df_indicateurs_pro = df_indicateurs_pro.astype(
        {
        'annee': 'category',
        'uai': 'category',
        'presents_toutes_series': pd.Int16Dtype(),
        'taux_de_reussite_toutes_series': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_de_reussite_toutes_series': pd.Int16Dtype(),
        'taux_d_acces_2nde_bac': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_d_acces_2nde_bac': pd.Int16Dtype(),
        'taux_d_acces_1ere_bac': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_d_acces_1ere_bac': pd.Int16Dtype(),
        'taux_d_acces_terminale_bac': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_d_acces_terminale_bac': pd.Int16Dtype(),
        'taux_de_mentions_toutes_series': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_de_mentions_toutes_series': pd.Int16Dtype()
        }
    )
except ValueError as erreur:
    print(erreur)

invalid literal for int() with base 10: '.'


Il reste des variables dont la valeur est « . ». On les remplace aussi par « NaN ».

In [44]:
df_indicateurs_gt = df_indicateurs_gt.replace('.', np.NaN)

df_indicateurs_gt = df_indicateurs_gt.astype(
    {
        'annee': 'category',
        'uai': 'category',
        'presents_toutes_series': pd.Int16Dtype(),
        'taux_de_reussite_toutes_series': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_de_reussite_toutes_series': pd.Int16Dtype(),
        'taux_d_acces_2nde_bac': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_d_acces_2nde_bac': pd.Int16Dtype(),
        'taux_d_acces_1ere_bac': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_d_acces_1ere_bac': pd.Int16Dtype(),
        'taux_d_acces_terminale_bac': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_d_acces_terminale_bac': pd.Int16Dtype(),
        'taux_de_mentions_toutes_series': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_de_mentions_toutes_series': pd.Int16Dtype()
    }
)

df_indicateurs_pro = df_indicateurs_pro.replace('.', np.NaN)

df_indicateurs_pro = df_indicateurs_pro.astype(
    {
        'annee': 'category',
        'uai': 'category',
        'presents_toutes_series': pd.Int16Dtype(),
        'taux_de_reussite_toutes_series': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_de_reussite_toutes_series': pd.Int16Dtype(),
        'taux_d_acces_2nde_bac': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_d_acces_2nde_bac': pd.Int16Dtype(),
        'taux_d_acces_1ere_bac': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_d_acces_1ere_bac': pd.Int16Dtype(),
        'taux_d_acces_terminale_bac': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_d_acces_terminale_bac': pd.Int16Dtype(),
        'taux_de_mentions_toutes_series': pd.Int16Dtype(),
        'valeur_ajoutee_du_taux_de_mentions_toutes_series': pd.Int16Dtype()
    }
)

Voyons les valeurs manquantes.

In [45]:
display(df_indicateurs_gt.isna().sum())
display(df_indicateurs_pro.isna().sum())

annee                                                0
uai                                                  0
presents_toutes_series                               0
taux_de_reussite_toutes_series                       0
valeur_ajoutee_du_taux_de_reussite_toutes_series    19
taux_d_acces_2nde_bac                               70
valeur_ajoutee_du_taux_d_acces_2nde_bac             93
taux_d_acces_1ere_bac                                1
valeur_ajoutee_du_taux_d_acces_1ere_bac             23
taux_d_acces_terminale_bac                           1
valeur_ajoutee_du_taux_d_acces_terminale_bac        23
taux_de_mentions_toutes_series                       0
valeur_ajoutee_du_taux_de_mentions_toutes_series    19
dtype: int64

annee                                                0
uai                                                  0
presents_toutes_series                               0
taux_de_reussite_toutes_series                       0
valeur_ajoutee_du_taux_de_reussite_toutes_series    21
taux_d_acces_2nde_bac                                1
valeur_ajoutee_du_taux_d_acces_2nde_bac             14
taux_d_acces_1ere_bac                                1
valeur_ajoutee_du_taux_d_acces_1ere_bac             14
taux_d_acces_terminale_bac                           1
valeur_ajoutee_du_taux_d_acces_terminale_bac        14
taux_de_mentions_toutes_series                       0
valeur_ajoutee_du_taux_de_mentions_toutes_series    21
dtype: int64

Etant donné le nombre d'enregistrement, il y a peu de valeurs manquantes. Sur la plateforme [data.education.gouv.fr](https://data.education.gouv.fr/pages/accueil/), il est d'ailleurs précisé :

*« Les taux d’accès et leur valeur ajoutée sont calculés uniquement pour les lycées qui offrent un cycle complet, c’est-à-dire qui accueillent des élèves de 2nde, 1ère et Tale. Enfin, les résultats en valeur ajoutée et les effectifs de lauréats par mention ne sont pas diffusés lorsqu’il y a moins de 20 candidats en série GT ou moins de 10 candidats en série PRO. »*

On concatène les deux *DataFrames* et on exporte le résultat en CSV. Comme l'étude porte sur la France métropolitaine on supprime également les enregistrements liés aux DROM-COM.

In [46]:
# Concatène les données
df_temp = pd.concat(
    [df_indicateurs_gt, df_indicateurs_pro]).reset_index(drop=True)

# Exporte en csv
df_temp.to_csv('./datasets/ival-lycees-2017-2022-hors-drom-com.csv',
                    index=False, decimal=',', quoting=1)

In [47]:
del(df_indicateurs_gt, df_indicateurs_pro, df_temp)